In [14]:
# Just so that you don't have to restart the notebook with every change.
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import scienceplots
from dimensionality_reduction import *
from PIL import Image

plt.style.use(["science", "ieee", "no-latex", "std-colors"])


def load_and_flatten_image(image_path):
    img = Image.open(image_path).resize((32, 32))  # Resize to a manageable size
    img_array = np.array(img).flatten()
    return img_array / 255.0  # Normalize to [0, 1]


# Function to plot and save results
def plot_and_save(result, method_name, dataset_name, labels) -> None:
    plt.figure(figsize=(6, 4))
    for class_dir in class_dirs:
        mask = labels == class_dir
        plt.scatter(result[mask, 0], result[mask, 1], label=class_dir, s=5)
    plt.legend(loc="center left", bbox_to_anchor=(1, 0.5), ncol=2, fontsize=14)
    # plt.title(f"{dataset_name} {method_name} Visualization")
    plt.savefig(f"{dataset_name}_{method_name.lower()}_plot.png", dpi=600)
    plt.close()


# ds_name = "merged_ds"
# ds_name = "guitar-chords-tiny"
ds_name = "guitar-chords-ours-A-G"

dataset_names = {
    "merged_ds": "Guitar-chords (Merged)",
    "guitar-chords-tiny": "Guitar-chords-tiny",
    "guitar-chords-ours-A-G": "Guitar-chords-ours",
}

dataset_real_name = dataset_names[ds_name]

path = f"/home/dhimitriosduka/Documents/UdS/SoSe 2024/High-Level Computer Vision/Assignments/Datasets/Videos/sampled/"
# path = os.path.join(os.getcwd(), "Used Datasets", ds_name)

# Directories
base_dirs = [os.path.join(path, "train")]
class_dirs = ["A", "B", "C", "D", "E", "F", "G", "Am", "Bm", "Cm", "Dm", "Em", "Fm", "Gm"]

In [17]:
# Collect data and labels
data = []
labels = []

for base_dir in base_dirs:
    for class_dir in class_dirs:
        dir_path = os.path.join(base_dir, class_dir)
        for filename in os.listdir(dir_path):
            if filename.endswith(".jpg"):
                image_path = os.path.join(dir_path, filename)
                img_data = load_and_flatten_image(image_path)
                data.append(img_data)
                labels.append(class_dir)

data = np.array(data)
labels = np.array(labels)

In [18]:
try_these_methods = ["t-SNE"]  # ["t-SNE", "PCA", "UMAP", "LDA"]

FUNCS = {
    "PCA": apply_pca,
    "ICA": apply_ica,
    "LDA": apply_lda,
    "t-SNE": apply_tsne,
    "UMAP": partial(apply_umap, n_neighbors=15, min_dist=0.1, metric="euclidean", n_jobs=-1),
}

for method in try_these_methods:
    func = FUNCS[method]
    if method == "LDA":
        result = func(data, labels)
    else:
        result = func(data)
    plot_and_save(result, method, dataset_real_name, labels)
    print(f"{method} done for {dataset_real_name}, saved as {dataset_real_name}_{method.lower()}_plot.png")

Applying t-SNE (perplexity=133, learning_rate=1108) using opentsne
t-SNE done for Guitar-chords-ours, saved as Guitar-chords-ours_t-sne_plot.png
